In [6]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import modules.holistic_module as hm
from tensorflow.keras.models import load_model
import math
from modules.utils import Vector_Normalization
from modules.utils import Merge_Jamo_With_LLM
from PIL import ImageFont, ImageDraw, Image


from transformers import MllamaForConditionalGeneration, MllamaProcessor
import torch
import requests


In [7]:
#설정
fontpath = "AppleGothic.ttf"
font = ImageFont.truetype(fontpath, 40)

actions = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ',
             'ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ',
             'ㅐ', 'ㅒ', 'ㅔ', 'ㅖ', 'ㅢ', 'ㅚ', 'ㅟ']
seq_length = 10

# MediaPipe holistic model
detector = hm.HolisticDetector(min_detection_confidence=0.3)

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="multi_hand_gesture_classifier.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


I0000 00:00:1742823176.145226 6107532 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Max


In [10]:
# korean bllossom model
model = MllamaForConditionalGeneration.from_pretrained(
    "Bllossom/llama-3.2-Korean-Bllossom-AICA-5B",
    torch_dtype=torch.bfloat16,
    device_map='auto')
processor = MllamaProcessor.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-AICA-5B")
url = "https://cdn.discordapp.com/attachments/1156141391798345742/1313407928287494164/E18489E185B3E1848FE185B3E18485E185B5E186ABE18489E185A3E186BA202021-11-1620E1848BE185A9E18492E185AE2011.png?ex=675005f4&is=674eb474&hm=fc9c4231203f53c27f6edd2420961c182dd4a1ed14d4b73e04127f11393729af&"
image = Image.open(requests.get(url, stream=True).raw)


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 3/3 [00:09<00:00,  3.10s/it]


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x3aad650e0>

In [ ]:

cap = cv2.VideoCapture(0)

seq = []
action_seq = []
last_action = None
zamo_list = ["",]

2025-03-17 17:23:51.861 Python[61882:2287504] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


In [ ]:

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    img = detector.findHolistic(img, draw=True)
    _, right_hand_lmList = detector.findRighthandLandmark(img)

    # 오른손이 감지되었을 때
    if right_hand_lmList is not None:

        joint = np.zeros((42, 2))
        
        # 오른손 랜드마크 리스트
        for j, lm in enumerate(right_hand_lmList.landmark):
            joint[j] = [lm.x, lm.y]

        # 벡터 정규화 : 백터, 앵글
        vector, angle_label = Vector_Normalization(joint)
    
        # 백터와 각도를 하나의 배열로 합침
        d = np.concatenate([vector.flatten(), angle_label.flatten()])
        

        seq.append(d)
        if len(seq) < seq_length: # 만약 시퀀스의 길이가 10보다 작다면 다음으로 넘어감
            continue
        
        # 시퀀스 데이터와 넘파이화
        # 시퀀스 데이터를 최근 10개로 자름, 입력으로 쓰기 위해 차원 확장
        input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0) 
        input_data = np.array(input_data, dtype=np.float32)

        # tflite 모델을 활용한 예측
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke() # 모델 실행
        y_pred = interpreter.get_tensor(output_details[0]['index']) # 예측 결과 가져오기
        i_pred = int(np.argmax(y_pred[0])) # 가장 확률이 높은 클래스
        conf = y_pred[0][i_pred] # 예측 결과의 확률 (신뢰도)

        if conf < 0.9: # 신뢰도가 90% 미만이면 무시
            continue

        action = actions[i_pred] # 라벨 -> 실제값
        action_seq.append(action) # 시퀀스에 추가

        if len(action_seq) < 3: # 시퀀스 길이가 3보다 작으면 다음 루프로 넘어감
            continue

        this_action = '?'
        if action_seq[-1] == action_seq[-2] == action_seq[-3]: # 연속 3번 같으면 현재 액션으로 확정
            this_action = action

            # if last_action != this_action: # 이전 액션과 현재 액션이 다르면 
            #     last_action = this_action
        
        # 기록된 한글 파악
        if zamo_list[-1] != this_action: # 만약 전에 기록된 글자와 이번 글자가 다르다면
            zamo_list.append(this_action)
            print("zamo list : ",zamo_list)
        zamo_str = ''.join(zamo_list) # 리스트에 있는 단어 합침
        unitl_action = Merge_Jamo_With_LLM(zamo_str) # 합친 단어 한글로 만들기
        
        
        # 한글 폰트 출력    
        img_pil = Image.fromarray(img)
        draw = ImageDraw.Draw(img_pil)
        draw.text((10, 30), f'{unitl_action.upper()}', font=font, fill=(255, 255, 255))

        img = np.array(img_pil)

        
        

    cv2.imshow('img', img)
    if cv2.waitKey(1) & 0xFF == 27:
        break


W0000 00:00:1742199833.702270 2291555 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2025-03-17 17:23:54.098 Python[61882:2287504] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-17 17:23:54.098 Python[61882:2287504] +[IMKInputSession subclass]: chose IMKInputSession_Modern
